In [ ]:
import random
from paho.mqtt import client as mqtt_client

import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


broker = 'test.mosquitto.org'
port = 1883
topic = "Room Temperature"
# generate client ID with pub prefix randomly
client_id = f'python-mqtt-{random.randint(0, 100)}'

ti = []
T = []

def connect_mqtt() -> mqtt_client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    #client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        temp = msg.payload.decode()
        print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
        ani = FuncAnimation(plt.gcf(), animate, interval=100)
        plt.tight_layout()
        plt.show()

    client.subscribe(topic)
    client.on_message = on_message

# This function is called periodically from FuncAnimation
def animate(i):
    #time.sleep(1)
    T.append(temp)
    ti.append(dt.datetime.now().strftime('%H:%M:%S.%f'))

    # Create figure for plotting
    plt.cla()
    plt.plot(ti, T)
    plt.xlabel("Time")
    plt.ylabel("Room Temperature (C)")
    plt.tight_layout()

def run():
    client = connect_mqtt()
    subscribe(client)
    client.loop_forever()


if __name__ == '__main__':
    run()